# Simulación
Dada la complejidad, los trabajadores de la empresesa decidieron desarrollar parte de su código en la plataforma "Notebook de python"

In [15]:
type Bolita = 
    {
        x: float
        y: float
        vx: float
        vy: float
    }


type Bloque = 
    {
        x: float // Posición x de esquina inferior izquierda
        y: float // Posición y de esquina inferior izquierda
        Lx: float
        Ly: float
    }


In [17]:
let condicion_inicial_bolita = 
    let bolita:Bolita = {
        x = 1.
        y = 1.
        vx = 0.
        vy = 1.
    }
    bolita



let condicion_inicial_bloques =
    let bloque:Bloque = {
        x = 0.
        y = 0.
        Lx = 10.
        Ly = 10.
    }
    [bloque;bloque] 

let eliminar_bloque (bloques:Bloque list) (bloque:Bloque) =
    bloques

let actualiza_bolita (bolita:Bolita) =
    bolita

In [19]:
//Paredes.fs


type Paredes =
    {
        Left_x: float
        Down_y: float
        Right_x: float
        Up_y: float
    }

let paredes_inicial:Paredes =
    {   
        Left_x = 0.
        Down_y = 0.
        Right_x = 100.
        Up_y = 100.
    }


let condicion_inicial_paredes = 
    paredes_inicial

In [14]:
//Barrita.fs

type Barra = 
    {
        x: float
        y: float
        L: float
    }

type MovimientoBarra = 
    | Izquierda
    | Derecha
    | Ninguno

let barra_inicial:Barra = 
    {
        x = 50.0
        y = 1.
        L = 5.
    }

let condicion_inicial_barra =
    barra_inicial

let actualizo_barra (barra:Barra) (movimiento:MovimientoBarra) (dx_barra:float) =
    match movimiento with
    | Izquierda -> {barra with x = barra.x - dx_barra}
    | Derecha -> {barra with x = barra.x + dx_barra}
    | Ninguno -> barra


In [16]:
//Interacciones.fs

let Interaccion_pared (paredes:Paredes) (bolita:Bolita) =
    match bolita with
    | bolita when bolita.x <= paredes.Left_x -> {bolita with vx = -bolita.vx}
    | bolita when bolita.x >= paredes.Right_x -> {bolita with vx = -bolita.vx}
    | bolita when bolita.y >= paredes.Up_y -> {bolita with vy = -bolita.vy}
    | bolita when bolita.y <= paredes.Down_y -> bolita
    | _ -> bolita

let Interaccion_barra (barra:Barra) (bolita:Bolita) =
    if bolita.y <= barra.y && bolita.x >= barra.x && bolita.x <= barra.x + barra.L then
        {bolita with vy = -bolita.vy}
    else 
        bolita

type Bloque_eliminado =
    | Eliminado of Bloque
    | NoEliminado

type Choque_con_bloque = 
    | Vertical
    | Horizontal

let choca_bloque (bolita:Bolita) (bloque:Bloque) = 
    bolita.y <= bloque.y + bloque.Ly && bolita.y >= bloque.y && bolita.x >= bloque.x && bolita.x <= bloque.x + bloque.Lx

let Tipo_choque (bolita:Bolita) (bloque:Bloque) =
    let distancias_a_bloque = [bolita.y - bloque.y; bloque.y + bloque.Ly - bolita.y; bolita.x - bloque.x; bloque.x + bloque.Lx - bolita.x]
    let minimo = List.min distancias_a_bloque
    if minimo = bolita.y - bloque.y || minimo = bloque.y + bloque.Ly - bolita.y then
        Vertical
    else
        Horizontal
        
let Interaccion_bloques (bloques: Bloque List) (bolita: Bolita) =

    let rec Iteracion_sobre_bloques (bloques: Bloque List) (bolita: Bolita) =
        match bloques with
        | [] -> (bolita, Bloque_eliminado.NoEliminado)
        | bloque::bloques -> 
            if choca_bloque bolita bloque then
                let tipo_choque = Tipo_choque bolita bloque
                match tipo_choque with
                    | Vertical -> ({bolita with vy = -bolita.vy}, Bloque_eliminado.Eliminado bloque)
                    | Horizontal -> ({bolita with vx = -bolita.vx}, Bloque_eliminado.Eliminado bloque)
            else
                Iteracion_sobre_bloques bloques bolita

    Iteracion_sobre_bloques bloques bolita



In [18]:
//End

let Bolita_escapa (bolita:Bolita) (pared:Paredes) : bool = 
    bolita.y < pared.Down_y

let Bloques_vacios (bloques:Bloque List) : bool =
    bloques = []

let termina_juego (bolita:Bolita) (pared:Paredes) (bloques:Bloque List) :bool =
    Bolita_escapa bolita pared || Bloques_vacios bloques
    

In [20]:
let interfaz (bolita:Bolita) (bloques:Bloque) (barra:Barra) =
    //Print sofisticado

    ()

In [21]:
let bolita:Bolita = condicion_inicial_bolita
let bloques:Bloque List = condicion_inicial_bloques
let barra:Barra = condicion_inicial_barra
let paredes:Paredes = condicion_inicial_paredes

let dt = 1e-3
let vx_barra = 1e-2

let mutable terminar_juego = false

while terminar_juego = false do

let rec loop_temporal (bolita:Bolita) (bloques:Bloque List) (barra:Barra) (paredes:Paredes) (terminar_juego:bool) = 

    let new_objects: Bolita * Bloque_eliminado = 
        actualiza_bolita bolita dt
        |> Interaccion_pared paredes
        |> Interaccion_barra barra
        |> Interaccion_bloques bloques

    let new_bolita: Bolita = fst new_objects
    let bloque_eliminado: Bloque_eliminado = snd new_objects
    let new_bloques: Bloque List = 
        match bloque_eliminado with
            | Bloque_eliminado.Eliminado bloque -> 
                eliminar_bloque bloques bloque
            |_ ->
                bloques


    //Actualizamos la barra
    
    let new_barra:Barra = actualizo_barra barra movimiento vx_barra dt

    // interfaz new_bolita4 new_bloques new_barra

    let terminar_juego:bool = termina_juego new_bolita paredes new_bloques

    if terminar_juego = false then
        loop_temporal new_bolita new_bloques new_barra paredes terminar_juego
    else
        terminar_juego

        
    System.Threading.Thread.Sleep(10)




Detenido debido a un error.


Error: input.fsx (13,1)-(13,4) parse error El bloque que sigue a este objeto 'let' está sin finalizar. Cada bloque de código es una expresión y debe tener un resultado. 'let' no puede ser el elemento de código final en un bloque. Considere la posibilidad de asignar a este bloque un resultado explícito.

In [22]:
let assert_equal (expected) (actual) = 
    if expected=actual then 
        printfn "Pasó el test" 
    else 
        printfn "Error en el test"

In [27]:
//Test de Barra

let barra:Barra = 
    {
        x = 50.0
        y = 1.
        L = 5.
    }

let movimiento1:MovimientoBarra = Izquierda
let movimiento2:MovimientoBarra = Derecha
let movimiento3:MovimientoBarra = Ninguno
let dx_barra = 1

let barra_after1:Barra = 
    {
        x = 49.0
        y = 1.
        L = 5.
    }

let barra_after2:Barra = 
    {
        x = 51.0
        y = 1.
        L = 5.
    }

let barra_after3:Barra = barra



let Test_actualizo_barra =
    assert_equal barra_after1 (actualizo_barra barra movimiento1 dx_barra)
    assert_equal barra_after2 (actualizo_barra barra movimiento2 dx_barra)
    assert_equal barra_after3 (actualizo_barra barra movimiento3 dx_barra)
    


Pasó el test
Pasó el test
Pasó el test


Detenido debido a un error.


Error: input.fsx (10,1)-(10,4) parse error Construcción estructurada incompleta en este punto o antes. en enlace. Se esperaba '=' u otro token.